In [ ]:
import Pkg
Pkg.activate(".")

In [ ]:
using Gridap
using GridapGmsh
using Gridap.Geometry

In [ ]:
function thermistor(msh, U₀, fname, Tw = 200.0, order=1, Tref=0.0)
    Prₐ = 0.707
    kₐ = 25.3e-3
    νₐ = 1.6e-5

    D₁ = 2e-3
    D₂ = 0.5e-3
    Re₁ = U₀ * D₁ / νₐ
    Re₂ = U₀ * D₂ / νₐ

    Nu₁ = 0.51 * Re₁^0.5 * Prₐ^0.37
    Nu₂ = 0.51 * Re₂^0.5 * Prₐ^0.37

    k₁ = 1.0
    k₂ = 50.0
    h₁ = kₐ * Nu₁ / D₁
    h₂ = kₐ * Nu₂ / D₂ 

    model = GmshDiscreteModel(msh)
    degree = 2*order
    reffe = ReferenceFE(lagrangian, Float64, order)
    V = TestFESpace(model, reffe, dirichlet_tags=["temp", "tempend"])
    U = TrialFESpace(V, [Tw-Tref, 0])

    Ω = Triangulation(model)
    dΩ = Measure(Ω, degree)    

    Γ₁ = BoundaryTriangulation(model, tags="hglass")
    dΓ₁ = Measure(Γ₁, degree)    
    
    Γ₂ = BoundaryTriangulation(model, tags="hsteel")
    dΓ₂ = Measure(Γ₂, degree)    
    
    labels = get_face_labeling(model)
    dimension = 2
    tags = get_face_tag(labels, dimension);    
    tag_steel = get_tag_from_name(labels, "steel")
    tag_glass = get_tag_from_name(labels, "glass")    
    
    heatflux = (∇u, tag) -> begin
        if tag == tag_steel
            return k₂ * ∇u
        else
            return k₁ * ∇u
        end 
    end
    
    f(x) = 0.0
    r(x) = x[2]

    a(u,v) = ∫( r*∇(v)⋅ (heatflux∘(∇(u), tags) ) )*dΩ + ∫(r*v*u*h₁)*dΓ₁ + ∫(r*v*u*h₂)*dΓ₂
    b(v) = ∫(r*v*f)*dΩ
    println("Tw = $Tw")
    println("Re₁ = $Re₁")
    println("Re₂ = $Re₂")
    println("h₁ = $h₁ W/(m²K)")
    println("h₂ = $h₂ W/(m²K)")    
    op = AffineFEOperator(a,b,U,V)
    
    ls = LUSolver()
    solver = LinearFESolver(ls)
    
    uh = solve(solver, op)
    
    writevtk(Ω, fname, cellfields=["T"=>uh+Tref])
    return uh, Ω, dΩ, Γ₁, dΓ₁, Γ₂, dΓ₂
end


In [ ]:
x = thermistor("mf58.msh", 20.0, "lixo")

In [ ]:
numstring(x, n=3) = string(x + 10^n)[2:end]

In [ ]:
U = [0.1, 0.2, 0.3, 0.5, 0.7, 1.0, 1.2, 1.5, 2.0, 2.5, 3.0, 3.5, 4, 5, 6,7, 8, 10, 12, 14, 16, 18, 20]
fnames = "thermistor_" .* numstring.(round.(Int, 10*U))
thermistor.("mf58.msh", U, fnames)

In [ ]:
x = thermistor("mf58.msh", 10.0, "teste_02", 200)

In [ ]:
g = ∇(x[1])

In [ ]:
typeof(x[1])

In [ ]:
supertypes(typeof(x[1]))

In [ ]:
typeof(x[1]) <: AbstractArray

In [ ]:
Map

In [ ]:
lagrangian

In [ ]:
order = 1
msh = "mf58.msh"
model = GmshDiscreteModel(msh)
degree = 2*order
reffe = ReferenceFE(lagrangian, Float64, order)
Tw = 200.0
Tref = 0.0
V = TestFESpace(model, reffe, dirichlet_tags="temp")
U = TrialFESpace(V, Tw-Tref)

Ω = Triangulation(model)
dΩ = Measure(Ω, degree)    

Γ₁ = BoundaryTriangulation(model, tags="hglass")
dΓ₁ = Measure(Γ₁, degree)    
    
Γ₂ = BoundaryTriangulation(model, tags="hsteel")
dΓ₂ = Measure(Γ₁, degree)    

In [ ]:
import Gridap.Geometry

In [ ]:
y = get_facet_normal(x[4]);

In [ ]:
typeof(y) <: AbstractArray

In [ ]:
z = y[1]

In [ ]:
z

In [ ]:
z.op

In [ ]:
uh = x[1]

In [ ]:
typeof(uh) <: CellField

In [ ]:
typeof(∫)

In [ ]:
∫

In [ ]:
methods(Gridap.CellData.Integrand)


In [ ]:
H = integrate(uh, x[5])

In [ ]:
(typeof(x[5])) <: CellQuadrature

In [ ]:
typeof(H)

In [ ]:
typeof(∫(uh)*x[5])

In [ ]:
H

In [ ]:
xx = H.dict


In [ ]:
xx.ndel


In [ ]:
using Gridap.CellData

In [ ]:

get_normal_vector(x[4]) * 

In [ ]:
get_cell_points(Ω)